In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import joblib

# 1. Cargar datos
df = pd.read_csv("MLP/dataset_entregas_bn_1000.csv")

target_col = "EntregaATiempo"
y = df[target_col].map({"No": 0, "Si": 1})
X = df.drop(columns=[target_col])

# 2. Columnas numéricas y categóricas
num_cols = X.select_dtypes(exclude=["object", "category"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

# 3. Preprocesamiento (se guarda dentro del pipeline)
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

# 4. Modelo (red neuronal)
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation="relu",
    solver="adam",
    max_iter=300,
    random_state=42,
)

# 5. Pipeline completo: preprocesa + entrena
pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", mlp),
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipe.fit(X_train, y_train)

# 6. Evaluación
y_pred = pipe.predict(X_test)
print(classification_report(y_test, y_pred))

# 7. Guardar pipeline para usar en el backend
joblib.dump(pipe, "modelo_entregas_mlp.pkl")


              precision    recall  f1-score   support

           0       0.88      0.86      0.87       101
           1       0.86      0.88      0.87        99

    accuracy                           0.87       200
   macro avg       0.87      0.87      0.87       200
weighted avg       0.87      0.87      0.87       200



c:\Users\alexi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


['modelo_entregas_mlp.pkl']